In [1]:
!pip install -q PyDrive
!pip install flair

     |████████████████████████████████| 337kB 9.1MB/s 
     |████████████████████████████████| 798kB 30.0MB/s 
     |████████████████████████████████| 983kB 17.1MB/s 
     |████████████████████████████████| 256kB 49.9MB/s 
     |████████████████████████████████| 757kB 48.1MB/s 
     |████████████████████████████████| 1.1MB 48.8MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 3.0MB 36.5MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25020 sha256=3f4ba161e2d3ee6bed9022237d64869e147c9fb78a1244edab134cd5331497f1
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=69d36873f4fe43e860ce0095a0ce97c1eeb691e07a20ffdcbbb2f7ca7795657f
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for langdetect: filenam

In [2]:
import copy
import logging
import numpy as np
import pandas as pd

from google.colab import drive
from flair.data import Sentence
from collections import defaultdict
from flair.models import TextClassifier

logging.basicConfig(level=logging.CRITICAL)

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
DIMENSION_DF = '/content/gdrive/My Drive/trendata/sentiment.csv'
IN_LOCATION = '/content/gdrive/My Drive/trendata/ALL-DATA-OUT-4.CSV'
OUT_LOCATION = '/content/gdrive/My Drive/trendata/ALL-DATA-OUTS-4.csv'

In [5]:
classifier = TextClassifier.load('en-sentiment')

2020-06-30 05:56:33,294 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert.pt not found in cache, downloading to /tmp/tmplocijk9s


100%|██████████| 266170364/266170364 [00:06<00:00, 40595118.87B/s]

2020-06-30 05:56:39,914 copying /tmp/tmplocijk9s to cache at /root/.flair/models/sentiment-en-mix-distillbert.pt


2020-06-30 05:56:41,356 removing temp file /tmp/tmplocijk9s
2020-06-30 05:56:43,132 loading file /root/.flair/models/sentiment-en-mix-distillbert.pt


In [6]:
ordering = ['negative opinion', 'positive opinion', 'value attribute', 'problems']

print('Reading dimensions and tags from {}.'.format(DIMENSION_DF))
sentiment_df = pd.read_csv(DIMENSION_DF, delimiter='\t')

print('Reading DATA from {}.'.format(IN_LOCATION))
review_df_orig = pd.read_csv(IN_LOCATION, delimiter='\t')

Reading dimensions and tags from /content/gdrive/My Drive/trendata/sentiment.csv.
Reading DATA from /content/gdrive/My Drive/trendata/ALL-DATA-OUT-4.CSV.


In [7]:
df_columns = list(sentiment_df.columns)
tag_index = df_columns.index('Tag')
regex_index = df_columns.index('Regex')
dimension_index = df_columns.index('Dimensie')

rule_to_dim, tag_to_dim = {}, {}
for row in sentiment_df.to_numpy().tolist():
    dimension, tag = row[dimension_index], row[tag_index]
    rules, stack = [], []
    for rule_component in row[regex_index:]:
        if rule_component and not pd.isna(rule_component):
            if rule_component != '|':
                if rule_component.startswith('{') and rule_component.endswith('}'):
                    rule_component = rule_component[1:-1]
                stack.append(rule_component)
            elif rule_component == '|' and len(stack):
                rules.append(' '.join(stack))
                stack = []
    if len(stack):
        rules.append(' '.join(stack))

    if dimension and not pd.isna(dimension) and tag and not pd.isna(tag):
        for rule in rules:
            rule_to_dim[rule.lower()] = (tag, dimension)
        tag_to_dim[tag] = dimension

In [8]:
_punctuation = '.?!'
_translate_map = str.maketrans(dict((c, '') for c in _punctuation))

print('Creating inverted index.')
keywords = []
inverted_index = defaultdict(set)
for i, row in enumerate(review_df_orig['Review (Original)'].to_numpy()):
    keywords.append(row)
    if isinstance(row, str):
        for word in row.lower().translate(_translate_map).split():
            inverted_index[word].add(i)

Creating inverted index.


In [9]:
_translate_map = str.maketrans(dict((c, c + '\n') for c in _punctuation))
index_to_tag = defaultdict(lambda: defaultdict(list))

index_to_matching_tags = defaultdict(set)
print('Looking up tags in data')
size = len(rule_to_dim)
for i, (rule, (tag, dim)) in enumerate(sorted(rule_to_dim.items(), key=lambda x: len(x[0]), reverse=True)):
    if not i % 1000:
        print('Progress: {}/{}'.format(i, size))
    if isinstance(rule, str):
        rule_split = rule.split()
        rule_doc_ids = set.intersection(*[inverted_index[t] for t in rule_split]) if len(rule_split) else set()

        for rule_doc_id in rule_doc_ids:
            paragraph = copy.deepcopy(keywords[rule_doc_id]).translate(_translate_map)
            for sentence in paragraph.splitlines():
                index = sentence.find(rule)
                if index >= 0 and not any(map(lambda t: tag in t, index_to_matching_tags[rule_doc_id])):
                    index_to_tag[rule_doc_id][tag].append(sentence)
                    index_to_matching_tags[rule_doc_id].add(tag)

Looking up tags in data
Progress: 0/13807
Progress: 1000/13807
Progress: 2000/13807
Progress: 3000/13807
Progress: 4000/13807
Progress: 5000/13807
Progress: 6000/13807
Progress: 7000/13807
Progress: 8000/13807
Progress: 9000/13807
Progress: 10000/13807
Progress: 11000/13807
Progress: 12000/13807
Progress: 13000/13807


In [10]:
doc_id_to_tag_to_sentiment = defaultdict(dict)

doc_id_to_positive = {}
doc_id_to_negative = {}
doc_id_to_value = {}
doc_id_to_problem = {}

sentiment = []
print('Determining sentiment of sentences surrounding tags.')
size = len(index_to_tag)

flair_sentences = []
for i, (doc_id, tags) in enumerate(index_to_tag.items()):
    if not i % 1000:
        print('Progress: {}/{}'.format(i, size))

    for tag, sentences in tags.items():
        for sentence in sentences:

            flair_sentence = Sentence(sentence)
            classifier.predict(flair_sentence)
            compound = (-flair_sentence.labels[0].score 
                        if flair_sentence.labels[0].value == 'NEGATIVE'
                        else flair_sentence.labels[0].score)

            if compound >= .75:
                tp = 'Positive'
                dimension = tag_to_dim[tag].lower()
                if dimension == ordering[0]:
                    if doc_id not in doc_id_to_negative or compound > doc_id_to_negative[doc_id][1]:
                        doc_id_to_negative[doc_id] = (tp, compound, tag)
                elif dimension == ordering[1]:
                    if doc_id not in doc_id_to_positive or compound > doc_id_to_positive[doc_id][1]:
                        doc_id_to_positive[doc_id] = (tp, compound, tag)
                elif dimension == ordering[2]:
                    if doc_id not in doc_id_to_value or compound > doc_id_to_value[doc_id][1]:
                        doc_id_to_value[doc_id] = (tp, compound, tag)
                elif dimension == ordering[3]:
                    if doc_id not in doc_id_to_problem or compound > doc_id_to_problem[doc_id][1]:
                        doc_id_to_problem[doc_id] = (tp, compound, tag)
                else:
                    print('NOT FOUND ' + dimension)
            elif compound <= -.75:
                tp = 'Negative'
                compound = np.abs(compound)
                dimension = tag_to_dim[tag].lower()
                if dimension == ordering[0]:
                    if doc_id not in doc_id_to_negative or compound > doc_id_to_negative[doc_id][1]:
                        doc_id_to_negative[doc_id] = (tp, compound, tag)
                elif dimension == ordering[1]:
                    if doc_id not in doc_id_to_positive or compound > doc_id_to_positive[doc_id][1]:
                        doc_id_to_positive[doc_id] = (tp, compound, tag)
                elif dimension == ordering[2]:
                    if doc_id not in doc_id_to_value or compound > doc_id_to_value[doc_id][1]:
                        doc_id_to_value[doc_id] = (tp, compound, tag)
                elif dimension == ordering[3]:
                    if doc_id not in doc_id_to_problem or compound > doc_id_to_problem[doc_id][1]:
                        doc_id_to_problem[doc_id] = (tp, compound, tag)
            else:
                tp = 'Neutral'
                compound = np.abs(compound)
                dimension = tag_to_dim[tag].lower()
                if dimension == ordering[0]:
                    if doc_id not in doc_id_to_negative or compound > doc_id_to_negative[doc_id][1]:
                        doc_id_to_negative[doc_id] = (tp, compound, tag)
                elif dimension == ordering[1]:
                    if doc_id not in doc_id_to_positive or compound > doc_id_to_positive[doc_id][1]:
                        doc_id_to_positive[doc_id] = (tp, compound, tag)
                elif dimension == ordering[2]:
                    if doc_id not in doc_id_to_value or compound > doc_id_to_value[doc_id][1]:
                        doc_id_to_value[doc_id] = (tp, compound, tag)
                elif dimension == ordering[3]:
                    if doc_id not in doc_id_to_problem or compound > doc_id_to_problem[doc_id][1]:
                        doc_id_to_problem[doc_id] = (tp, compound, tag)

Determining sentiment of sentences surrounding tags.
Progress: 0/262058
Progress: 1000/262058
Progress: 2000/262058
Progress: 3000/262058
Progress: 4000/262058
Progress: 5000/262058
Progress: 6000/262058
Progress: 7000/262058
Progress: 8000/262058
Progress: 9000/262058
Progress: 10000/262058
Progress: 11000/262058
Progress: 12000/262058
Progress: 13000/262058
Progress: 14000/262058
Progress: 15000/262058
Progress: 16000/262058
Progress: 17000/262058
Progress: 18000/262058
Progress: 19000/262058
Progress: 20000/262058
Progress: 21000/262058
Progress: 22000/262058
Progress: 23000/262058
Progress: 24000/262058
Progress: 25000/262058
Progress: 26000/262058
Progress: 27000/262058
Progress: 28000/262058
Progress: 29000/262058
Progress: 30000/262058
Progress: 31000/262058
Progress: 32000/262058
Progress: 33000/262058
Progress: 34000/262058
Progress: 35000/262058
Progress: 36000/262058
Progress: 37000/262058
Progress: 38000/262058
Progress: 39000/262058
Progress: 40000/262058
Progress: 41000/2

In [11]:
ts1 = [None] * len(review_df_orig)
ts1t = [None] * len(review_df_orig)
ts2 = [None] * len(review_df_orig)
ts2t = [None] * len(review_df_orig)

for doc_id, v in doc_id_to_negative.items():
    if ts1[doc_id] is None:
        ts1[doc_id] = v[0]
        ts1t[doc_id] = v[2]
    elif ts2[doc_id] is None:
        ts2[doc_id] = v[0]
        ts2t[doc_id] = v[2]

for doc_id, v in doc_id_to_positive.items():
    if ts1[doc_id] is None:
        ts1[doc_id] = v[0]
        ts1t[doc_id] = v[2]
    elif ts2[doc_id] is None:
        ts2[doc_id] = v[0]
        ts2t[doc_id] = v[2]

for doc_id, v in doc_id_to_value.items():
    if ts1[doc_id] is None:
        ts1[doc_id] = v[0]
        ts1t[doc_id] = v[2]
    elif ts2[doc_id] is None:
        ts2[doc_id] = v[0]
        ts2t[doc_id] = v[2]

for doc_id, v in doc_id_to_problem.items():
    if ts1[doc_id] is None:
        ts1[doc_id] = v[0]
        ts1t[doc_id] = v[2]
    elif ts2[doc_id] is None:
        ts2[doc_id] = v[0]
        ts2t[doc_id] = v[2]

review_df_orig['Tag1'] = ts1t
review_df_orig['Tag1 Sentiment'] = ts1
review_df_orig['Tag2'] = ts2t
review_df_orig['Tag2 Sentiment'] = ts2

print('Writing result')
review_df_orig.to_csv(OUT_LOCATION, sep='\t', index=False)

Writing result
